In [1]:
import requests
import json

x_auth_token = '9bf7641735e6561359dd5bba2017cc07'
base_url = 'https://kox947ka1a.execute-api.ap-northeast-2.amazonaws.com/prod/users'


header = {'X-Auth-Token':x_auth_token, 'Content-Type': 'application/json'}
data = {'problem':2}

res = requests.post(base_url+"/start",data=json.dumps(data), headers=header)

auth_key = res.json()['auth_key']


In [2]:

def getInfo(st):
    #locations, trucks, score
    h = {'Authorization':auth_key, 'Content-Type': 'application/json' }
    return requests.get(base_url+"/"+st,headers= h).json()

def simulate(command):
    h = {'Authorization':auth_key, 'Content-Type': 'application/json' }
    data = {"commands": command}
    return requests.put(base_url+"/simulate",data=json.dumps(data), headers=h).json()

NO = 0
UP = 1
RIGHT = 2
DOWN = 3
LEFT = 4
LOAD = 5
UNLOAD = 6
MAXSIZE = 60

## Most-To-Least
def goAtoB(a,b):
    command = []
    a_row = a//MAXSIZE
    a_col = a%MAXSIZE
    b_row = b//MAXSIZE
    b_col = b%MAXSIZE
    move = 0
    if a_row < b_row:
        move=RIGHT
    else:
        move=LEFT    
    for i in range(abs(b_row - a_row)):
        command.append(move)
    
    if a_col < b_col:
        move=UP
    else:
        move=DOWN 
    for i in range(abs(b_col - a_col)):
        command.append(move)
        
    return command

def getMost(tid):
    tmp = loc_arr[MAXSIZE*tid*6:MAXSIZE*(tid+1)*6]
    return tmp.index(max(tmp)) + MAXSIZE*tid*6, max(tmp)
    
def getLeast(tid):
    tmp = loc_arr[MAXSIZE*tid*6:MAXSIZE*(tid+1)*6]
    return tmp.index(min(tmp)) + MAXSIZE*tid*6, min(tmp)
    
def setLoc():
    arr = []
    for v in getInfo("locations")['locations']:
        arr.append(v['located_bikes_count'])
    return arr

def setTruckLoc():
    loc_arr = []
    bike_arr = []
    for v in getInfo("trucks")['trucks']:
        loc_arr.append(v['location_id'])
        bike_arr.append(v['loaded_bikes_count'])
    return loc_arr,bike_arr

def moveToBase():
    cmd = []
    for i in range(10):
        cmd.append(makeCommand(i, goAtoB(truck_loc_arr[i], i*MAXSIZE*6)))
    return cmd



def makeCommand(tid,command):
    return {"truck_id":tid, "command":command}


In [3]:
def cmdTruck(tid,target,load,t):
    command = []
    command.extend(goAtoB(truck_loc_arr[tid], target))
    for _ in range(t):
        command.append(load)
    return command

def doLoad(tid):
    im, vm = getMost(tid)
    il, vl = getLeast(tid)
    t = 0
    if vm-vl > 3:
        t=2
    elif vm-vl >= 2:
        t=1
    command = []
    command.extend(goAtoB(truck_loc_arr[tid], im))
    for _ in range(t):
        command.append(LOAD)
    
    command.extend(goAtoB(im, il))
    for _ in range(t):
        command.append(UNLOAD)
    return makeCommand(tid,command)
    


def doUnload(tid):
    command = []
    il,vl = getLeast(tid)
    command.extend(goAtoB(truck_loc_arr[tid], il))
    for _ in range(2):
        command.append(UNLOAD)
    
    return makeCommand(tid,command)
    

In [4]:
truck_loc_arr, truck_bike_arr = setTruckLoc()
loc_arr = setLoc() 
simulate(moveToBase())

{'status': 'ready',
 'time': 1,
 'failed_requests_count': '0.0',
 'distance': '8.6'}

In [5]:
from tqdm import tqdm
s = ""
for _ in tqdm(range(719)):
    cmd = []
    truck_loc_arr, truck_bike_arr = setTruckLoc()
    loc_arr = setLoc()    
    for tid in range(10):
        if truck_bike_arr[tid]!=0:
            cmd.append(doUnload(tid))
        else:
            cmd.append(doLoad(tid))
    s = simulate(cmd)

print(s)

100%|██████████| 719/719 [04:58<00:00,  2.41it/s]

{'status': 'finished', 'time': 720, 'failed_requests_count': '556.0', 'distance': '6205.5'}


In [6]:
10829-556

10273

In [7]:
print(getInfo("score"))

{'score': 635.6908714447318}
